In [1]:
# The following preliminary analyses come from a Visual Search Task performed by two different samples: older vs younger adults.

# In this particular task: 
# the set size of the stimuli presented varied according to 3 levels (5, 7 or 10 stimuli in total displayed);
# the difficulty of the visual search also varied according to 3 levels 
### (Baseline, where stimuli could come from any conceptual category -such as kitchen materials, tools, clothing, ...- and any color category -such as blue, brown, black,...-;
### High Concept, where stimuli could come only from one conceptual category but could be in different colors;
### High Color, where stimuli could only be in one color but could come from different conceptual categories);
# the Target that had to be searched for could be present or not.

# Both Reaction Time (RT) and accuracy were measured.

import numpy as np
import pandas as pd
import random
import itertools
from scipy.stats import ttest_ind  # LP: very minor but to keep code consistent you could have used stats.ttest_ind (or explicitely import all function like here)
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns

In [ ]:
# STEP 1. generation of some data

# ensure reproducibility by putting a seed
np.random.seed(42)
random.seed(42)


# LP: Veru nice definition of variables at top level; you could have done this for all variable though, and avoid
# all numbers in the code loop below!
# Also, you could have split the code into functions taking careof different parts of the data generation separately for readability
# define variables levels
Nfig_levels = [5, 7, 10]  
Tipo_levels = ['Baseline', 'Hconc', 'Hcol'] 
TargetPresence_levels = ['T', 'NoT']
Group_levels = ['OLD', 'YOUNG']

# define the sample size
n_samples = 30

# define the number of participants per group
n_participants_per_group = 30

# define the total number of participants
n_participants_tot = n_participants_per_group * len(Group_levels)

# define the total number of trials for each combination of conditions
total_trials_per_condition = 40

# create a function for the data generation according to some boundaries
def generate_ideal_data():
    data = []
    # initialize participant ID
    participant_id = 1 
    
    # loop through each group level (YOUNG and OLD)
    for group in Group_levels:
        # loop through each participant within the group
        for _ in range(n_participants_per_group):
            # loop through each combination of Nfig, Tipo, and TargetPresence levels (so that it will generate RT and accuracy based on these combinations)
            for Nfig, Tipo, TargetPresence in itertools.product(Nfig_levels, Tipo_levels, TargetPresence_levels): # itertools.product to generate all possible combinations of the levels of the independent variables
                # define the baseline of RT according to the group
                if group == 'YOUNG':
                    base_rt = 0.71
                else:  # Group == 'OLD'
                    base_rt = 0.89
                # YOUNG are faster than OLD
                    
                # generate RT as a function of the predictors
                rt_nfig = (Nfig - 5) * 1.13  # increase by 1.45 for each additional level of difficulty compared to the easier level
                rt_tipo = 0.91 if Tipo == 'Baseline' else (1.09 if Tipo == 'Hconc' else 1.2) # adjusting RT based on Tipo
                rt_target = 0.8 if TargetPresence == 'T' else 1.56 # adjusting RT based on TargetPresence
                rt_group = 0.97 if group == 'YOUNG' else 1.1 # adjusting RT based on Group
                
                # apply the baseline value
                rt = base_rt * (1 + (rt_nfig + rt_tipo + rt_target + rt_group) / 100)
                
                # Add interaction effects (e.g., Tipo might interact with TargetPresence)
                if Tipo != 'Baseline' and TargetPresence == 'NoT':
                    rt += 0.15  # Increase RT for non-baseline tasks when target is not present

                # Add some random noise for RT, with higher noise for more difficult tasks
                difficulty = rt_nfig + rt_tipo + rt_target + rt_group
                noise_level = 0.3 + 0.1 * (difficulty / 50)  # Increase noise with difficulty
                rt += np.random.normal(0, noise_level)
                
                # add some random noise for RT
                #rt += random.normalvariate(0, 0.3)
                
                # boundaries for RT (ex. no negative values but also not too close to 0)
                rt = max(0.15, min(rt, 4))
                
                # Add small random buffer to avoid hitting the exact boundaries
                if rt == 0.15:
                    rt += np.random.uniform(0.01, 0.02)  # Slightly increase to avoid the lower boundary
                if rt == 4:
                    rt -= np.random.uniform(0.01, 0.02)  # Slightly decrease to avoid the upper boundary

                # the resulting RT will be the mean RT for each participant for each condition combination
                    
                # define the baseline of Accuracy according to the group
                if group == 'YOUNG':
                    base_accuracy = 0.87
                else:  # Group == 'OLD'
                    base_accuracy = 0.96
                # OLD are more accurate than YOUNG
                
                # generate accuracy as a function of the predictors
                acc_nfig = -0.01 if Nfig == 5 else (-0.09 if Nfig == 7 else -0.13)  # adjust accuracy based on Nfig (decreasing with higher Nfig)
                acc_tipo = 0.02 if Tipo == 'Baseline' else (-0.03 if Tipo == 'Hconc' else -0.06) # adjust accuracy based on Tipo
                acc_target = 0 if TargetPresence == 'NoT' else -0.05 # adjust accuracy based on TargetPresence
                acc_group = 0.03 if group == 'OLD' else -0.03 # adjust accuracy based on Group
                
                # apply the baseline value
                accuracy = base_accuracy + acc_nfig + acc_tipo + acc_target + acc_group
                
                # Add interaction effects (e.g., Tipo might interact with TargetPresence)
                if Tipo != 'Baseline' and TargetPresence == 'T':
                    accuracy -= 0.02  # Decrease accuracy for non-baseline tasks when target is present

                # Add some random noise for Accuracy, with higher noise for more difficult tasks
                difficulty = abs(acc_nfig) + abs(acc_tipo) + abs(acc_target) + abs(acc_group)
                noise_level = 0.05 + 0.02 * (difficulty / 0.5)  # Increase noise with difficulty
                accuracy += np.random.normal(0, noise_level)
                
                # add some random noise for Accuracy
                #accuracy += random.normalvariate(0, 0.3)
                
                # boundaries for Accuracy 0.65 e 1
                accuracy = max(0.65, min(1, accuracy))
                
                # Add small random buffer to avoid hitting the exact boundaries
                if accuracy == 0.65:
                    accuracy += np.random.uniform(0.01, 0.02)  # Slightly increase to avoid the lower boundary
                
                # the resulting Accuracy will be the mean accuracy for each participant for each condition combination
                
                # calculate the number of correct trials based on accuracy
                correct_trials = round(accuracy * total_trials_per_condition)
                
                data.append([participant_id, Nfig, Tipo, TargetPresence, group, rt, accuracy, correct_trials])
            
            participant_id += 1
    
    return pd.DataFrame(data, columns=['ID', 'Nfig', 'Tipo', 'TargetPresence', 'Group', 'RT', 'Accuracy', 'correct_trials'])

# generate data
df_ideal = generate_ideal_data()

print(df_ideal.head(50))
print(df_ideal.tail(50))

# Save to an Excel file
df_ideal.to_excel('ideal_data.xlsx', index=False)

In [ ]:
# STEP 2. T-test RT

# t-test mean RT OLD vs YOUNG not considering any predictor
rt_old = df_ideal[df_ideal['Group'] == 'OLD']['RT']
rt_young = df_ideal[df_ideal['Group'] == 'YOUNG']['RT']

t_stat, p_value = ttest_ind(rt_old, rt_young)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

In [ ]:
# STEP 3. visualization of mean RT OLD vs YOUNG not considering any predictor

# LP: Not bad code but you could have reduce it by exactly half defining functions for stats and plots (or a loop) and then iterated over the two groups!
# Also, see comment in the next cells for jittered data points plotting 

# calculate mean RT and standard error for YOUNG and OLD
mean_rt_young = df_ideal[df_ideal['Group'] == 'YOUNG']['RT'].mean()
std_err_young = df_ideal[df_ideal['Group'] == 'YOUNG']['RT'].sem()
mean_rt_old = df_ideal[df_ideal['Group'] == 'OLD']['RT'].mean()
std_err_old = df_ideal[df_ideal['Group'] == 'OLD']['RT'].sem() 
print(f"YOUNG mean RT: {mean_rt_young}, YOUNG SE: {std_err_young}")
print(f"OLD mean RT: {mean_rt_old}, OLD SE: {std_err_old} ")

# mean RT for each participant considering all conditions together
mean_rt_young_ID = df_ideal[df_ideal['Group'] == 'YOUNG'].groupby('ID')['RT'].mean()
mean_rt_old_ID = df_ideal[df_ideal['Group'] == 'OLD'].groupby('ID')['RT'].mean()

# create a BAR plot
plt.figure(figsize=(10, 6))
plt.bar(['YOUNG', 'OLD'], [mean_rt_young, mean_rt_old], yerr=[std_err_young, std_err_old], capsize=5, color=['blue', 'violet'], label='Mean RT with SE')

# in order to visualize also single data points, create a jitter for each participant's data point
jitter_y = np.random.normal(loc=0, scale=0.05, size=len(mean_rt_young_ID))
jitter_o = np.random.normal(loc=0, scale=0.05, size=len(mean_rt_old_ID))

# add individual participant RTs as jittered data points
plt.scatter(np.zeros(len(mean_rt_young_ID)) + jitter_y, mean_rt_young_ID, color='pink', alpha=0.5, label='Individual RT (YOUNG)')
plt.scatter(np.ones(len(mean_rt_old_ID)) + jitter_o, mean_rt_old_ID, color='lightblue', alpha=0.5, label='Individual RT (OLD)')

# following the t-test results, add significance star if any
if p_value < 0.001:
    sig = '***'  # Highly significant
elif p_value < 0.01:
    sig = '**'  # Very significant
elif p_value < 0.05:
    sig = '*'  # Significant
else:
    sig = 'n.s.'  # Not significant
    
# position of the star or "n.s." is midway of the bars, slightly above the highest error bar
plt.text(0.5, max(mean_rt_young + std_err_young, mean_rt_old + std_err_old) + 0.15, sig, ha='center', va='bottom', color='black', fontsize=20)

# add line connecting the bars
y_max = max(mean_rt_young + std_err_young, mean_rt_old + std_err_old) + 0.15
plt.plot([0, 1], [y_max, y_max], color='black')

plt.xlabel('Group')
plt.ylabel('Mean Reaction Time (RT)')
plt.title('Mean Reaction Time (RT) Comparison between YOUNG and OLD')
plt.ylim(0.5, 1.4)  # Set y-axis range
plt.xticks([0, 1], ['YOUNG', 'OLD'])
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.show()

# create a BOXPLOT
plt.figure(figsize=(10, 6))
data = [mean_rt_young_ID, mean_rt_old_ID]
plt.boxplot(data, labels=['YOUNG', 'OLD'], patch_artist=True, boxprops=dict(facecolor='lightgrey', color='black', alpha=0.5), medianprops=dict(color='red'))

# In order to visualize also single data points, create a jitter for each participant's data point
jitter_y = np.random.normal(loc=1, scale=0.05, size=len(mean_rt_young_ID))
jitter_o = np.random.normal(loc=2, scale=0.05, size=len(mean_rt_old_ID))

# Add individual participant RTs as jittered data points
plt.scatter(jitter_y, mean_rt_young_ID, color='pink', alpha=0.5, label='Individual RT (YOUNG)')
plt.scatter(jitter_o, mean_rt_old_ID, color='lightblue', alpha=0.5, label='Individual RT (OLD)')

# Following the t-test results, add significance star if any
if p_value < 0.001:
    sig = '***'  # Highly significant
elif p_value < 0.01:
    sig = '**'  # Very significant
elif p_value < 0.05:
    sig = '*'  # Significant
else:
    sig = 'n.s.'  # Not significant

# Position of the star or "n.s." is midway of the boxes, slightly above the highest box
plt.text(1.5, max(mean_rt_young + std_err_young, mean_rt_old + std_err_old) + 0.15, sig, ha='center', va='bottom', color='black', fontsize=20)

# Add line connecting the boxes
y_max = max(mean_rt_young + std_err_young, mean_rt_old + std_err_old) + 0.15
plt.plot([1, 2], [y_max, y_max], color='black')

plt.xlabel('Group')
plt.ylabel('Mean Reaction Time (RT)')
plt.title('Mean Reaction Time (RT) Comparison between YOUNG and OLD')
plt.ylim(0.5, 1.4)  # Set y-axis range
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.show()

In [ ]:
# STEP 4. checking assumptions for parametric methods

# Assumption 1: Normality of the residuals
shapiro_test_rt = stats.shapiro(df_ideal['RT'])
print(f"Shapiro-Wilk test for RT: statistic = {shapiro_test_rt.statistic}, p-value = {shapiro_test_rt.pvalue}")

# KIND AND CURIOUS QUESTION FOR WHO IS CURRENTLY LOOKING AT MY CODE:
# HOW DO I MAKE IT POSSIBLE FOR THE RESIDUALS OF THE MODEL OF MY IDEAL DATA TO FOLLOW A NORMAL DISTRIBUTION?
# LP: If you really want normal noise an optionis to generate "pure" datapoints with no randomness and then add noise from known distribution.
# Not sure if this is what you want though!

# MOREOVER, IN R I WOULD USE THE Aligned Rank Transform (ART) AS A NON-PARAMETRIC TECHNIQUE USED FOR FACTORIAL DATA ANALYSIS THROUGH THE PACKAGE ARTool BUT I DIDN'T FIND SOMETHING SIMILAR FOR PYTHON
# WHAT SHOULD BE DONE IN MY CASE THAN?
# LP: see below

# Assumption 2: Homogeneity of variance
levene_test = stats.levene(df_ideal['RT'][df_ideal['Group'] == 'YOUNG'],
                            df_ideal['RT'][df_ideal['Group'] == 'OLD'])
print(f"Levene's test statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")

In [ ]:
# LP: # I do not know enough statistics to be of any help here, but here's an implementation that latest GPT offered, maybe it is worth compare the results with what you have in R :D
# Please ping me with the results if you try this out.
import pandas as pd
import numpy as np
from scipy.stats import rankdata
import statsmodels.api as sm
from statsmodels.formula.api import ols
from itertools import combinations

def aligned_rank_transform(data, dependent_var, factors, interaction_order=None):
    """
    Perform Aligned Rank Transform for Nonparametric Factorial ANOVAs,
    including interactions up to the specified order.

    Parameters:
    - data: pandas DataFrame containing the data.
    - dependent_var: string, name of the dependent variable.
    - factors: list of strings, names of the independent factors.
    - interaction_order: int or None, maximum order of interactions to include.
                         If None, include all possible interactions.

    Returns:
    - art_data: DataFrame with aligned and ranked data.
    """
    art_data = data.copy()
    art_data['aligned'] = 0.0  # Initialize aligned data

    # Generate all terms (main effects and interactions)
    if interaction_order is None:
        interaction_order = len(factors)
    terms = []
    for order in range(1, interaction_order + 1):
        terms += ['*'.join(combo) for combo in combinations(factors, order)]

    # Fit additive model with all specified interactions
    formula = dependent_var + ' ~ ' + ' + '.join(terms)
    model = ols(formula, data=data).fit()
    aligned_values = model.fittedvalues

    # Compute residuals
    residuals = data[dependent_var] - aligned_values

    # Add residuals to the grand mean to get aligned data
    grand_mean = data[dependent_var].mean()
    art_data['aligned'] = grand_mean + residuals

    # Rank the aligned data
    art_data['rank'] = art_data['aligned'].rank(method='average')

    return art_data

def anova_on_art(data, dependent_var='rank', factors=[], interaction_order=None):
    """
    Perform ANOVA on the Aligned Rank Transformed data, including interactions.

    Parameters:
    - data: pandas DataFrame containing the ART-transformed data.
    - dependent_var: string, name of the ranked dependent variable.
    - factors: list of strings, names of the independent factors.
    - interaction_order: int or None, maximum order of interactions to include.
                         Should match the order used in alignment.

    Returns:
    - anova_table: ANOVA results.
    """
    # Generate all terms (main effects and interactions)
    if interaction_order is None:
        interaction_order = len(factors)
    terms = []
    for order in range(1, interaction_order + 1):
        terms += ['*'.join(combo) for combo in combinations(factors, order)]

    # Construct the formula for ANOVA
    formula = dependent_var + ' ~ ' + ' * '.join(factors)
    print(formula)
    model = ols(formula, data=data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    return anova_table

# Example Usage

# Sample Data Creation
np.random.seed(0)
data = pd.DataFrame({
    'A': np.repeat(['Low', 'High'], 60),
    'B': np.tile(np.repeat(['Control', 'Treatment'], 30), 2),
    'Y': np.concatenate([
        np.random.normal(10, 2, 30),
        np.random.normal(12, 2, 30),
        np.random.normal(15, 2, 30),
        np.random.normal(17, 2, 30)
    ])
})

# Apply Aligned Rank Transform with Interaction
factors = ['Group', 'TargetPresence', 'Nfig', 'Tipo']
interaction_order = 2  # For two-way interactions
art_data = aligned_rank_transform(df_ideal, 'RT', factors, interaction_order=interaction_order)

# Perform ANOVA on Ranked Data
anova_results = anova_on_art(art_data, dependent_var='RT', factors=factors, interaction_order=interaction_order)

print("Aligned Rank Transform ANOVA Results:")
print(anova_results)

In [ ]:
# STEP 5. ANOVA (or non-parametric test)

# create a formula for the ANOVA
formula = 'RT ~ C(Group) + C(Nfig) + C(Tipo) + C(TargetPresence) + C(Group):C(Nfig) + C(Group):C(Tipo) + C(Group):C(TargetPresence) + C(Group):C(Nfig):C(Tipo) + C(Group):C(Nfig):C(TargetPresence) + C(Group):C(Tipo):C(TargetPresence)'

model = ols(formula, df_ideal).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

In [ ]:
# STEP 6. POST-HOC tests
# LP: You know more stats than I do for sure :)

# perform post-hoc test for 'Group'
posthoc_group = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Group'], alpha=0.05)
print(posthoc_group)

# perform post-hoc test for 'Nfig'
posthoc_nfig = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Nfig'], alpha=0.05)
print(posthoc_nfig)

# perform post-hoc test for 'Tipo'
posthoc_tipo = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Tipo'], alpha=0.05)
print(posthoc_tipo)

# perform post-hoc test for 'TargetPresence'
posthoc_target_presence = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['TargetPresence'], alpha=0.05)
print(posthoc_target_presence)

# create interaction columns
# LP: Probably the best here would have been a loop over pairs of interactor columns to avoid repetition
df_ideal['Group_Nfig'] = df_ideal['Group'] + "_" + df_ideal['Nfig'].astype(str)
df_ideal['Group_Tipo'] = df_ideal['Group'] + "_" + df_ideal['Tipo']
df_ideal['Group_TargetPresence'] = df_ideal['Group'] + "_" + df_ideal['TargetPresence']
df_ideal['Group_Nfig_Tipo'] = df_ideal['Group'] + "_" + df_ideal['Nfig'].astype(str) + "_" + df_ideal['Tipo']
df_ideal['Group_Nfig_TargetPresence'] = df_ideal['Group'] + "_" + df_ideal['Nfig'].astype(str) + "_" + df_ideal['TargetPresence']
df_ideal['Group_Tipo_TargetPresence'] = df_ideal['Group'] + "_" + df_ideal['Tipo'] + "_" + df_ideal['TargetPresence']

# post-hoc test for 'Group:Nfig' interaction
posthoc_group_nfig = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Group_Nfig'], alpha=0.05)
print(posthoc_group_nfig)

# post-hoc test for 'Group:Tipo' interaction
posthoc_group_tipo = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Group_Tipo'], alpha=0.05)
print(posthoc_group_tipo)

# post-hoc test for 'Group:TargetPresence' interaction
posthoc_group_target_presence = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Group_TargetPresence'], alpha=0.05)
print(posthoc_group_target_presence)

# post-hoc test for 'Group:Nfig:Tipo' interaction
posthoc_group_nfig_tipo = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Group_Nfig_Tipo'], alpha=0.05)
print(posthoc_group_nfig_tipo)

# post-hoc test for 'Group:Nfig:TargetPresence' interaction
posthoc_group_nfig_target_presence = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Group_Nfig_TargetPresence'], alpha=0.05)
print(posthoc_group_nfig_target_presence)

# post-hoc test for 'Group:Tipo:TargetPresence' interaction
posthoc_group_tipo_target_presence = pairwise_tukeyhsd(df_ideal['RT'], df_ideal['Group_Tipo_TargetPresence'], alpha=0.05)
print(posthoc_group_tipo_target_presence)

In [ ]:
# STEP 7. visualization of mean RT OLD vs YOUNG considering predictors interactions

# LP: nice but no need for dynamite plots! :D
# Also, very nice that you dag so much into plotting with jittering and all the alike but I would have written a specific function to deal with it!
# It's likely you can need that code elsewhere in the future! :)

# calculate the mean RT for each possible interaction of all levels of the conditions
for Nfig in Nfig_levels:
    for Tipo in Tipo_levels:
        for TargetPresence in TargetPresence_levels:
            for Group in Group_levels:
                # filter the dataframe based on the current levels
                df_filtered = df_ideal[(df_ideal['Nfig'] == Nfig) &
                                                      (df_ideal['Tipo'] == Tipo) &
                                                      (df_ideal['TargetPresence'] == TargetPresence) &
                                                      (df_ideal['Group'] == Group)]
                
                # calculate the mean RT and standard error
                mean_rt = df_filtered['RT'].mean()
                std_err = df_filtered['RT'].sem()
                
                # Print the results
                print(f"Mean RT for Nfig={Nfig}, Tipo={Tipo}, TargetPresence={TargetPresence}, Group={Group}: {mean_rt}, SE: {std_err}")

# initialize a dictionary to store the jitter for each participant
jitter_dict = {}

# calculate the jitter for each possible interaction of all levels of the conditions
for Nfig in Nfig_levels:
    for Tipo in Tipo_levels:
        for TargetPresence in TargetPresence_levels:
            for Group in Group_levels:
                # filter the dataframe based on the current levels
                df_filtered = df_ideal[(df_ideal['Nfig'] == Nfig) &
                                                      (df_ideal['Tipo'] == Tipo) &
                                                      (df_ideal['TargetPresence'] == TargetPresence) &
                                                      (df_ideal['Group'] == Group)]
                
                # get the unique participant IDs
                IDs = df_filtered['ID'].unique()
                
                # calculate the jitter for each participant
                for ID in IDs:
                    jitter = np.random.normal(loc=0, scale=0.05)
                    
                    # store the jitter in the dictionary
                    jitter_dict[(Nfig, Tipo, TargetPresence, Group, ID)] = jitter
                    
# create a new column that combines the Tipo and Nfig levels
df_ideal['Tipo_Nfig'] = df_ideal['Tipo'] + '_' + df_ideal['Nfig'].astype(str)

# create a dictionary to map Group to color
color_dict = {'YOUNG': 'blue', 'OLD': 'violet'}
color_dict2 = {'YOUNG': 'pink', 'OLD': 'lightblue'}

# create two separate plots according to TargetPresence
for TargetPresence in TargetPresence_levels:
    # filter the DataFrame based on the current TargetPresence
    df_filtered = df_ideal[df_ideal['TargetPresence'] == TargetPresence]
    
    # create a bar plot with error bars
    plt.figure(figsize=(15, 8))
    sns.barplot(data=df_filtered, x='Tipo_Nfig', y='RT', hue='Group', palette=color_dict, capsize=.1)
    
    # get the unique 'Tipo_Nfig' values
    unique_Tipo_Nfig = df_filtered['Tipo_Nfig'].unique()
    
    # add individual participant mean RT as jittered data points
    for Group in Group_levels:
        for i, Tipo_Nfig in enumerate(unique_Tipo_Nfig):
            # filter the dataframe based on the current 'Tipo_Nfig' and 'Group'
            df_filtered_Tipo_Nfig_Group = df_filtered[(df_filtered['Tipo_Nfig'] == Tipo_Nfig) & (df_filtered['Group'] == Group)]
            
            # get the unique participant IDs
            IDs = df_filtered_Tipo_Nfig_Group['ID'].unique()
            
            # plot the mean RT for each participant
            for ID in IDs:
                # introduce a horizontal offset for both 'Young' and 'Old' groups
                offset = -0.2 if Group == 'OLD' else 0.2
                # marker = 'x' if Group == 'YOUNG' else 'o'
                plt.scatter(i + offset + jitter_dict[(Nfig, Tipo, TargetPresence, Group, ID)], df_filtered_Tipo_Nfig_Group[df_filtered_Tipo_Nfig_Group['ID'] == ID]['RT'].mean(), color=color_dict2[Group], alpha=0.5) #, marker=marker)
    
    plt.title(f'Mean Reaction Time (RT) for each condition (TargetPresence = {TargetPresence})')
    plt.xlabel('Condition (Tipo_Nfig)')
    plt.ylabel('Mean Reaction Time (RT)')
    
    # show the plot
    plt.show()
    
# create two separate BOXPLOTS according to TargetPresence
for TargetPresence in TargetPresence_levels:
    # filter the dataframe based on the current TargetPresence
    df_filtered = df_ideal[df_ideal['TargetPresence'] == TargetPresence]
    
    # set size of the plot
    plt.figure(figsize=(15, 8)) 
    # create a box plot with jittered data points
    sns.boxplot(data=df_filtered, x='Tipo_Nfig', y='RT', hue='Group', palette=color_dict)
    
    
    # get the unique 'Tipo_Nfig' values
    unique_Tipo_Nfig = df_filtered['Tipo_Nfig'].unique()
    
    # add individual participant mean RT as jittered data points
    for Group in Group_levels:
        for i, Tipo_Nfig in enumerate(unique_Tipo_Nfig):
            # filter the dataframe based on the current 'Tipo_Nfig' and 'Group'
            df_filtered_Tipo_Nfig_Group = df_filtered[(df_filtered['Tipo_Nfig'] == Tipo_Nfig) & (df_filtered['Group'] == Group)]
            
            # get the unique participant IDs
            IDs = df_filtered_Tipo_Nfig_Group['ID'].unique()
            
            # plot the mean RT for each participant
            for ID in IDs:
                # introduce a horizontal offset for both 'Young' and 'Old' groups
                offset = -0.2 if Group == 'OLD' else 0.2
                # marker = 'x' if Group == 'YOUNG' else 'o'
                plt.scatter(i + offset + jitter_dict[(Nfig, Tipo, TargetPresence, Group, ID)], df_filtered_Tipo_Nfig_Group[df_filtered_Tipo_Nfig_Group['ID'] == ID]['RT'].mean(), color=color_dict2[Group], alpha=0.5) #, marker=marker) 
    
    plt.ylim(0, 2)  # Set y-axis range
    plt.title(f'Mean Reaction Time (RT) for each condition (TargetPresence = {TargetPresence})')
    plt.xlabel('Condition (Tipo_Nfig)')
    plt.ylabel('Mean Reaction Time (RT)')
    
    # show the plot
    plt.show()

In [ ]:
# STEP 8. T-test Accuracy    
    
# t-test mean accuracy OLD vs YOUNG without considering predictors
accuracy_old = df_ideal[df_ideal['Group'] == 'OLD']['Accuracy']
accuracy_young = df_ideal[df_ideal['Group'] == 'YOUNG']['Accuracy']

t_stat, p_value = ttest_ind(accuracy_old, accuracy_young)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

In [ ]:
# STEP 9. visualization of mean Accuracy OLD vs YOUNG without considering predictors
# LP: for example here !
# Also, there is a huge amount of code duplication, and you are actually doing the same thing as above. Try to refactor it into a function!
# your very nice code for adding significance could also go in a function 

# calculate mean accuracy + standard error for YOUNG & OLD
mean_acc_young = df_ideal[df_ideal['Group'] == 'YOUNG']['Accuracy'].mean()
std_err_acc_young = df_ideal[df_ideal['Group'] == 'YOUNG']['Accuracy'].sem()
mean_acc_old = df_ideal[df_ideal['Group'] == 'OLD']['Accuracy'].mean()
std_err_acc_old = df_ideal[df_ideal['Group'] == 'OLD']['Accuracy'].sem() 
print(f"YOUNG mean Accuracy: {mean_acc_young}, YOUNG SE: {std_err_acc_young}")
print(f"OLD mean Accuracy: {mean_acc_old}, OLD SE: {std_err_acc_old} ")

# mean accuracy of each participant
mean_acc_young_ID = df_ideal[df_ideal['Group'] == 'YOUNG'].groupby('ID')['Accuracy'].mean()
mean_acc_old_ID = df_ideal[df_ideal['Group'] == 'OLD'].groupby('ID')['Accuracy'].mean()

# create a BAR plot
plt.figure(figsize=(10, 6))
plt.bar(['YOUNG', 'OLD'], [mean_acc_young, mean_acc_old], yerr=[std_err_acc_young, std_err_acc_old], capsize=5, color=['blue', 'violet'], label='Mean Accuracy with SE')

# create jitter for each participant data point
jitter_y = np.random.normal(loc=0, scale=0.05, size=len(mean_acc_young_ID))
jitter_o = np.random.normal(loc=0, scale=0.05, size=len(mean_acc_old_ID))

# add individual participant accuracy as jittered data points
plt.scatter(np.zeros(len(mean_acc_young_ID)) + jitter_y, mean_acc_young_ID, color='pink', alpha=0.5, label='Individual Accuracy (YOUNG)')
plt.scatter(np.ones(len(mean_acc_old_ID)) + jitter_o, mean_acc_old_ID, color='lightblue', alpha=0.5, label='Individual Accuracy (OLD)')

# add significance star
if p_value < 0.001:
    sig = '***'  # Highly significant
elif p_value < 0.01:
    sig = '**'  # Very significant
elif p_value < 0.05:
    sig = '*'  # Significant
else:
    sig = 'n.s.'  # Not significant
    
# position of the star or "n.s." is midway of the bars, slightly above the highest error bar
plt.text(0.5, max(mean_acc_young + std_err_acc_young, mean_acc_old + std_err_acc_old) + 0.1, sig, ha='center', va='bottom', color='black', fontsize=20)

# add line connecting the bars
y_max = max(mean_acc_young + std_err_acc_young, mean_acc_old + std_err_acc_old) + 0.1
plt.plot([0, 1], [y_max, y_max], color='black')

plt.xlabel('Group')
plt.ylabel('Mean Accuracy')
plt.title('Mean Accuracy Comparison between YOUNG and OLD')
plt.ylim(0.6, 1)  # Set y-axis range
plt.xticks([0, 1], ['YOUNG', 'OLD'])
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.show()


# create a BOXPLOT
plt.figure(figsize=(10, 6))
data_acc = [mean_acc_young_ID, mean_acc_old_ID]
plt.boxplot(data_acc, labels=['YOUNG', 'OLD'], patch_artist=True, boxprops=dict(facecolor='lightgrey', color='black', alpha=0.5), medianprops=dict(color='red'))

# In order to visualize also single data points, create a jitter for each participant's data point
jitter_y = np.random.normal(loc=1, scale=0.05, size=len(mean_acc_young_ID))
jitter_o = np.random.normal(loc=2, scale=0.05, size=len(mean_acc_old_ID))

# Add individual participant RTs as jittered data points
plt.scatter(jitter_y, mean_acc_young_ID, color='pink', alpha=0.5, label='Individual RT (YOUNG)')
plt.scatter(jitter_o, mean_acc_old_ID, color='lightblue', alpha=0.5, label='Individual RT (OLD)')

# Following the t-test results, add significance star if any
if p_value < 0.001:
    sig = '***'  # Highly significant
elif p_value < 0.01:
    sig = '**'  # Very significant
elif p_value < 0.05:
    sig = '*'  # Significant
else:
    sig = 'n.s.'  # Not significant

# position of the star or "n.s." is midway of the bars, slightly above the highest error bar
plt.text(1.5, max(mean_acc_young + std_err_acc_young, mean_acc_old + std_err_acc_old) + 0.1, sig, ha='center', va='bottom', color='black', fontsize=20)

# add line connecting the bars
y_max = max(mean_acc_young + std_err_acc_young, mean_acc_old + std_err_acc_old) + 0.1
plt.plot([1, 2], [y_max, y_max], color='black')

plt.xlabel('Group')
plt.ylabel('Mean Accuracy')
plt.title('Mean Accuracy Comparison between YOUNG and OLD')
plt.ylim(0.6, 1)  # Set y-axis range
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.show()

In [ ]:
# STEP 10. checking assumptions for parametric methods

# Assumption 1: Normality
shapiro_test_accuracy = stats.shapiro(df_ideal['Accuracy'])
print(f"Shapiro-Wilk test for Accuracy: statistic = {shapiro_test_accuracy.statistic}, p-value = {shapiro_test_accuracy.pvalue}")

# Assumption 2: Homogeneity of variance
levene_test_acc = stats.levene(df_ideal['Accuracy'][df_ideal['Group'] == 'YOUNG'],
                            df_ideal['Accuracy'][df_ideal['Group'] == 'OLD'])
print(f"Levene's test statistic: {levene_test_acc.statistic}, p-value: {levene_test_acc.pvalue}")

In [ ]:
# STEP 11. ANOVA (or non-parametric test)

# create a formula for the ANOVA
formula2 = 'Accuracy ~ C(Group) + C(Nfig) + C(Tipo) + C(TargetPresence) + C(Group):C(Nfig) + C(Group):C(Tipo) + C(Group):C(TargetPresence) + C(Group):C(Nfig):C(Tipo) + C(Group):C(Nfig):C(TargetPresence) + C(Group):C(Tipo):C(TargetPresence)'

model2 = ols(formula2, df_ideal).fit()
anova_table2 = sm.stats.anova_lm(model2, typ=2)

print(anova_table2)

In [ ]:
# STEP 12. POST-HOC tests

# See comment above for duplication

# perform post-hoc test for 'Group'
posthoc_group = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Group'], alpha=0.05)
print(posthoc_group)

# perform post-hoc test for 'Nfig'
posthoc_nfig = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Nfig'], alpha=0.05)
print(posthoc_nfig)

# perform post-hoc test for 'Tipo'
posthoc_tipo = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Tipo'], alpha=0.05)
print(posthoc_tipo)

# perform post-hoc test for 'TargetPresence'
posthoc_target_presence = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['TargetPresence'], alpha=0.05)
print(posthoc_target_presence)

# post-hoc test for 'Group:Nfig' interaction
posthoc_group_nfig = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Group_Nfig'], alpha=0.05)
print(posthoc_group_nfig)

# post-hoc test for 'Group:Tipo' interaction
posthoc_group_tipo = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Group_Tipo'], alpha=0.05)
print(posthoc_group_tipo)

# post-hoc test for 'Group:TargetPresence' interaction
posthoc_group_target_presence = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Group_TargetPresence'], alpha=0.05)
print(posthoc_group_target_presence)

# post-hoc test for 'Group:Nfig:Tipo' interaction
posthoc_group_nfig_tipo = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Group_Nfig_Tipo'], alpha=0.05)
print(posthoc_group_nfig_tipo)

# post-hoc test for 'Group:Nfig:TargetPresence' interaction
posthoc_group_nfig_target_presence = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Group_Nfig_TargetPresence'], alpha=0.05)
print(posthoc_group_nfig_target_presence)

# post-hoc test for 'Group:Tipo:TargetPresence' interaction
posthoc_group_tipo_target_presence = pairwise_tukeyhsd(df_ideal['Accuracy'], df_ideal['Group_Tipo_TargetPresence'], alpha=0.05)
print(posthoc_group_tipo_target_presence)

In [ ]:
# STEP 13. visualization of mean Accuracy OLD vs YOUNG considering predictors interactions

# calculate the mean Accuracy for each possible interaction of all levels of the conditions
for Nfig in Nfig_levels:
    for Tipo in Tipo_levels:
        for TargetPresence in TargetPresence_levels:
            for Group in Group_levels:
                # filter the dataframe based on the current levels
                df_filtered = df_ideal[(df_ideal['Nfig'] == Nfig) &
                                                      (df_ideal['Tipo'] == Tipo) &
                                                      (df_ideal['TargetPresence'] == TargetPresence) &
                                                      (df_ideal['Group'] == Group)]
                
                # calculate the mean RT and standard error
                mean_acc = df_filtered['Accuracy'].mean()
                std_err_acc = df_filtered['Accuracy'].sem()
                
                # print the results
                print(f"Mean Accuracy for Nfig={Nfig}, Tipo={Tipo}, TargetPresence={TargetPresence}, Group={Group}: {mean_acc}, SE: {std_err_acc}")

# create two separate BAR plots according to TargetPresence
for TargetPresence in TargetPresence_levels:
    # filter the dataframe based on the current TargetPresence
    df_filtered = df_ideal[df_ideal['TargetPresence'] == TargetPresence]
    
    # create a bar plot with error bars
    plt.figure(figsize=(15, 8))
    sns.barplot(data=df_filtered, x='Tipo_Nfig', y='Accuracy', hue='Group', palette=color_dict, capsize=.1)
    
    # get the unique 'Tipo_Nfig' values
    unique_Tipo_Nfig = df_filtered['Tipo_Nfig'].unique()
    
    # add individual participant mean RT as jittered data points
    for Group in Group_levels:
        for i, Tipo_Nfig in enumerate(unique_Tipo_Nfig):
            # filter the dataframe based on the current 'Tipo_Nfig' and 'Group'
            df_filtered_Tipo_Nfig_Group = df_filtered[(df_filtered['Tipo_Nfig'] == Tipo_Nfig) & (df_filtered['Group'] == Group)]
            
            # get the unique participant IDs
            IDs = df_filtered_Tipo_Nfig_Group['ID'].unique()
            
            # plot the mean RT for each participant
            for ID in IDs:
                # introduce a horizontal offset for both 'Young' and 'Old' groups
                offset = -0.2 if Group == 'OLD' else 0.2
                # marker = 'x' if Group == 'YOUNG' else 'o'
                plt.scatter(i + offset + jitter_dict[(Nfig, Tipo, TargetPresence, Group, ID)], df_filtered_Tipo_Nfig_Group[df_filtered_Tipo_Nfig_Group['ID'] == ID]['Accuracy'].mean(), color=color_dict2[Group], alpha=0.5) #, marker=marker) 
    
    plt.ylim(0.6, 1.1)  # Set y-axis range
    plt.title(f'Mean Accuracy for each condition (TargetPresence = {TargetPresence})')
    plt.xlabel('Condition (Tipo_Nfig)')
    plt.ylabel('Mean Accuracy')
    
    # show the plot
    plt.show()
    
# create two separate BOXPLOTS according to TargetPresence
for TargetPresence in TargetPresence_levels:
    # filter the dataframe based on the current TargetPresence
    df_filtered = df_ideal[df_ideal['TargetPresence'] == TargetPresence]
    
    # set size of the plot
    plt.figure(figsize=(15, 8)) 
    # create a box plot with jittered data points
    sns.boxplot(data=df_filtered, x='Tipo_Nfig', y='Accuracy', hue='Group', palette=color_dict)
    
    
    # get the unique 'Tipo_Nfig' values
    unique_Tipo_Nfig = df_filtered['Tipo_Nfig'].unique()
    
    # add individual participant mean RT as jittered data points
    for Group in Group_levels:
        for i, Tipo_Nfig in enumerate(unique_Tipo_Nfig):
            # filter the dataframe based on the current 'Tipo_Nfig' and 'Group'
            df_filtered_Tipo_Nfig_Group = df_filtered[(df_filtered['Tipo_Nfig'] == Tipo_Nfig) & (df_filtered['Group'] == Group)]
            
            # get the unique participant IDs
            IDs = df_filtered_Tipo_Nfig_Group['ID'].unique()
            
            # plot the mean RT for each participant
            for ID in IDs:
                # introduce a horizontal offset for both 'Young' and 'Old' groups
                offset = -0.2 if Group == 'OLD' else 0.2
                # marker = 'x' if Group == 'YOUNG' else 'o'
                plt.scatter(i + offset + jitter_dict[(Nfig, Tipo, TargetPresence, Group, ID)], df_filtered_Tipo_Nfig_Group[df_filtered_Tipo_Nfig_Group['ID'] == ID]['Accuracy'].mean(), color=color_dict2[Group], alpha=0.5) #, marker=marker) 
    
    plt.ylim(0.6, 1.1)  # Set y-axis range
    plt.title(f'Mean Accuracy for each condition (TargetPresence = {TargetPresence})')
    plt.xlabel('Condition (Tipo_Nfig)')
    plt.ylabel('Mean Accuracy')
    
    # show the plot
    plt.show()